In [ ]:
#importando as bibliotecas usadas no projeto

import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score 
import torch 
import torch.nn as nn


definindo nossas bases de dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('/content/entradas_breast.csv')#dados das features
classe = pd.read_csv('/content/saidas_breast.csv')#dados de saida

In [ ]:
sns.countplot(classe['0']); #utilizando o grafico para verificar a relação entres 0 e 1 no conjunto de saida

In [ ]:
#dividindo os conjuntos previsores e classe em subconjuntos que serão usados para treinamento e para teste
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores,
                                                                                              classe,
                                                                                              test_size = 0.25)

Transformando os dados em tensores para poder usar no modelo

In [ ]:
previsores_treinamento = torch.tensor(np.array(previsores_treinamento), dtype=torch.float)
classe_treinamento = torch.tensor(np.array(classe_treinamento), dtype = torch.float)

In [ ]:
#visualizando se foi convertido
type(previsores_treinamento)

torch.Tensor

In [ ]:
type(classe_treinamento)

torch.Tensor

In [ ]:
#cria nosso dataset de tensores usando os tensores de previsores e de saida (classe)
dataset = torch.utils.data.TensorDataset(previsores_treinamento, classe_treinamento)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True) #usado para carregar nosso conjunto de dados de forma simples
#batch_size, que denota o número de amostras contidas em cada lote gerado.
#shuffle. Se definido como True, obteremos uma nova ordem de exploração a cada passagem (ou apenas manteremos um esquema de exploração linear caso contrário).

Construindo nosso modelo de treinamento

In [ ]:
#Como temos 30 colunas de features, usaremos 30 neuronios de entrada, usando 16 neuronios em cada hidden layer e diminuindo para um neuronio na camada de saida
classificador = nn.Sequential(
    nn.Linear(in_features=30, out_features=16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

In [ ]:
classificador.parameters #verificando se os parametros foram implementados corretamente

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=30, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=1, bias=True)
  (5): Sigmoid()
)>

In [ ]:
criterion = nn.BCELoss() #definindo a função de perda

In [ ]:
#metodo para atualizar os hiperparametros que podem reduzir as perdas
optimizer = torch.optim.Adam(classificador.parameters(), lr=0.001, weight_decay=0.0001)

Treinamento do modelo

In [ ]:
for epoch in range(100): #definindo quantas vezes será feito um forward pass e um backward pass(loop)
  running_loss = 0.

  for data in train_loader:#para cada dado no nosso dataset
    inputs, labels = data 
    #print(inputs)
    #print('-----')
    #print(labels)
    optimizer.zero_grad() #zera os parametros gradientes

    outputs = classificador(inputs)
    #print(outputs)
    loss = criterion(outputs, labels) #computa a perda
    #print(loss)
    loss.backward()#faz o backpropagation com a perda
    optimizer.step()#ajusta os parametros baseado nos calculos

    running_loss += loss.item()
  print('Época %3d: perda %.5f' % (epoch+1, running_loss/len(train_loader)))

Visualização dos pesos

In [ ]:
params = list(classificador.parameters())
#coloca os parametros em uma lista

In [ ]:
params #printa os parametros

In [ ]:
pesos0 = params[0]
pesos0.shape

In [ ]:
print(pesos0)

In [ ]:
bias0 = params[1]
bias0.shape

In [ ]:
pesos1 = params[2]
pesos1.shape

In [ ]:
bias1 = params[3]
bias1.shape

Avaliando o modelo

In [ ]:
classificador.eval()#Define o modelo no modo de avaliação (inferência)

In [ ]:
previsores_teste = torch.tensor(np.array(previsores_teste), dtype=torch.float)

In [ ]:
type(previsores_teste)

torch.Tensor

In [ ]:
# “encadeia” as saídas às entradas sequencialmente para cada módulo, retornando a saida do ultimo modulo
previsoes = classificador.forward(previsores_teste)

In [ ]:
previsoes

In [ ]:
previsoes = np.array(previsoes > 0.5) #coloca em um array as previsoes que foram maior que 0.5 (como true ou false)
previsoes

In [ ]:
#definindo a taxa de acerto do modelo comparando a classe de teste e as previsoes
taxa_acerto = accuracy_score(classe_teste, previsoes)
taxa_acerto

In [ ]:
#gera a matriz de confusao
matriz = confusion_matrix(classe_teste, previsoes)
matriz

In [ ]:
#gera um mapa de calor a partir da matriz
sns.heatmap(matriz, annot=True);